### Сгрузим данные из таблицы info

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import numpy as np
import pandas as pd

C:\Users\isobo\AppData\Local\Temp\ipykernel_15692\3354113047.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"] #права на работу с данной таблицей (readonly)

# The ID and range of a sample spreadsheet.
#Напишем функцию для выделения из ссылки на таблицу ее ID
def link_id(link):
    # Поиск подстроки "/d/", которая следует за "spreadsheets.google.com"
    start_idx = link.find("/d/")
    # Находим индекс конца идентификатора таблицы
    end_idx = link.find("/", start_idx + 3)
    # Извлекаем идентификатор таблицы
    sheet_id = link[start_idx + 3:end_idx]
    return sheet_id


def info(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES 
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
        values = result.get("values", [])

        if not values:
            print("No data found.")
            return

        # Извлечение заголовков столбцов
        headers = values[0]

        # Создание словаря для хранения данных по столбцам
        data_dict = {header: [] for header in headers}

        # Заполнение словаря данными из столбцов
        for row in values[1:]:
            for idx, value in enumerate(row):
                data_dict[headers[idx]].append(value)

        # заменим номер у каждой ссылки листа в конце на 0
        def replace_gid(link):
            # Разделение ссылки на две части по параметру gid
            parts = link.split("gid=")
            # Обновление значения параметра gid на 0
            new_link = parts[0] + "gid=0"
            return new_link
        
                # заменим номер у каждой ссылки листа в конце на 0
        for idx, link in enumerate(data_dict['Ссылка на общую таблицу контракта']):
            data_dict['Ссылка на общую таблицу контракта'][idx] = replace_gid(link)
        
        
        print(data_dict)

    except HttpError as err:
        print(err)
    return(data_dict)





In [3]:
if __name__ == "__main__":
    SAMPLE_SPREADSHEET_ID_INFO = "1MhYj1NjS3uv7gOC5JGp0wToC2YNH-6lg_inNSVo3K4o" #ID таблицы info (часть ссылки)
    SAMPLE_RANGE_NAME_INFO = "Сontract info шаблон!A1:E" #задаем лист и рассматриваемые ячейки
    info_dict = info(SAMPLE_SPREADSHEET_ID_INFO, SAMPLE_RANGE_NAME_INFO)

{'Название контракта': ['Контракт №1', 'Контракт №1', 'Контракт №1', 'Контракт №1', 'Контракт №2', 'Контракт №2'], 'Ссылка на общую таблицу контракта': ['https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1IUpVTWvWELWM9nGRMyxPKQ8XNBK1YCo0d7mYRW3PFIo/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1IUpVTWvWELWM9nGRMyxPKQ8XNBK1YCo0d7mYRW3PFIo/edit#gid=0'], 'Менеджер контракта': ['Блогеры ищеек', 'Nexus', 'Rahul', 'Крупные интеграции', 'Блогеры ищеек', 'Nexus'], 'Ссылка на лист контракта по типу менеджера исполнителя': ['https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gi

### сгрузим данные из таблиц всех из инфо

In [4]:
#Создадим словарь, с которого будем подгружать данные в таблицу Total
aggregate_tables = {'Ссылка на аккаунт исполнителя':[], 'Тип менеджера исполнителя':[], 'тип контракта':[], 'тг id менеджера исполнителя':[], 'Кол-во исполнителей':[], 'Дата выхода интеграции':[],'Расходы в Rs':[], 'трек ссылка':[]}


In [5]:

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"] #права на работу с данной таблицей (readonly)

#Напишем функцию для выделения из ссылки на таблицу ее ID
def link_id(link):
    # Поиск подстроки "/d/", которая следует за "spreadsheets.google.com"
    start_idx = link.find("/d/")
    # Находим индекс конца идентификатора таблицы
    end_idx = link.find("/", start_idx + 3)
    # Извлекаем идентификатор таблицы
    sheet_id = link[start_idx + 3:end_idx]
    return sheet_id


def table_lists(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES 
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
        values = result.get("values", [])

        if not values:
            print("No data found.")
            return

        # Извлечение заголовков столбцов
        headers = values[0]

        # Создание словаря для хранения данных по столбцам
        data_dict = {header: [] for header in headers}

        # Заполнение словаря данными из столбцов
        for row in values[1:]:
            for idx, value in enumerate(row):
                data_dict[headers[idx]].append(value)
        
        
        

    except HttpError as err:
        print(err)
    return(data_dict)





In [6]:
for elem in range(len(info_dict['Ссылка на общую таблицу контракта'])):

    if info_dict['Менеджер контракта'][elem] == 'Блогеры ищеек':
        #блогеры ищеек
        link = info_dict['Ссылка на общую таблицу контракта'][elem]
        sheet_id = link_id(link)
        range_name = info_dict['Менеджер контракта'][0] +'!A1:E'  # Указываем правильный диапазон данных
        table = table_lists(sheet_id, range_name)
        print(table)    
        aggregate_tables['Ссылка на аккаунт исполнителя'].append(table['Ссылка на аккаунт исполнителя'])
        aggregate_tables['Тип менеджера исполнителя'].append('Блогеры ищеек')
        aggregate_tables['тип контракта'].append(info_dict['Название контракта'][elem])
        aggregate_tables['тг id менеджера исполнителя'].append(table['тг id менеджера исполнителя'])
        aggregate_tables['Кол-во исполнителей'].append(1)
        aggregate_tables['Дата выхода интеграции'].append(table['Дата выхода интеграции'])
        aggregate_tables['Расходы в Rs'].append(table['Расходы в Rs'])
        aggregate_tables['трек ссылка'].append(table['трек ссылка'])

    elif info_dict['Менеджер контракта'][elem] == 'Крупные интеграции':
        # крупные интеграции
        link = info_dict['Ссылка на общую таблицу контракта'][elem]
        sheet_id = link_id(link)
        range_name = info_dict['Менеджер контракта'][elem] +'!A1:E'  # Указываем правильный диапазон данных
        table = table_lists(sheet_id, range_name)
        print(table)
        aggregate_tables['Ссылка на аккаунт исполнителя'].append(table['Ссылка на аккаунт исполнителя'])
        aggregate_tables['Тип менеджера исполнителя'].append('Крупные интеграции')
        aggregate_tables['тип контракта'].append(info_dict['Название контракта'][elem])
        aggregate_tables['тг id менеджера исполнителя'].append(table['тг id менеджера исполнителя'])
        aggregate_tables['Кол-во исполнителей'].append(1)
        aggregate_tables['Дата выхода интеграции'].append(table['Дата выхода интеграции'])
        aggregate_tables['Расходы в Rs'].append(table['Расходы в Rs'])
        aggregate_tables['трек ссылка'].append(table['трек ссылка'])

    elif not info_dict['Менеджер контракта'][elem] == 'Крупные интеграции' and  not info_dict['Менеджер контракта'][elem] == 'Блогеры ищеек':
        #агентства
        link = info_dict['Ссылка на общую таблицу контракта'][elem]
        sheet_id = link_id(link)
        range_name = info_dict['Менеджер контракта'][elem] +'!A1:D'  # Указываем правильный диапазон данных
        table = table_lists(sheet_id, range_name)
        print(table)

        aggregate_tables['Ссылка на аккаунт исполнителя'].append(np.nan)
        aggregate_tables['Тип менеджера исполнителя'].append('Агентства')
        aggregate_tables['тип контракта'].append(info_dict['Название контракта'][elem])
        aggregate_tables['тг id менеджера исполнителя'].append(table['тг id менеджера исполнителя'])
        aggregate_tables['Кол-во исполнителей'].append(table['Кол-во исполнителей'])
        aggregate_tables['Дата выхода интеграции'].append(np.nan)
        aggregate_tables['Расходы в Rs'].append(table['Расходы в Rs'])
        aggregate_tables['трек ссылка'].append(table['трек ссылка'])

{'Ссылка на аккаунт исполнителя': ['https://www.instagram.com/zakhmi.writess_?igsh=dDlzdTY4aHgzMHU3'], 'тг id менеджера исполнителя': ['999999999999'], 'Дата выхода интеграции': ['3/25/2024 13:40:08'], 'Расходы в Rs': ['7864287'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr&sub_id3=h257']}
{'тг id менеджера исполнителя': ['77777777'], 'Кол-во исполнителей': ['1672'], 'Расходы в Rs': ['1800000'], 'трек ссылка': ['ссылка нексус 1']}
{'тг id менеджера исполнителя': ['8888888888'], 'Кол-во исполнителей': ['1089'], 'Расходы в Rs': ['786890'], 'трек ссылка': ['ссылка рахул 1']}
{'Ссылка на аккаунт исполнителя': ['https://www.instagram.com/the_challengers07?igsh=dWdsdmxndnZpcDVm'], 'тг id менеджера исполнителя': ['10000000'], 'Дата выхода интеграции': ['22.05.2024'], 'Расходы в Rs': ['9878759'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr&sub_id3=h98']}
{'Ссылка на аккаунт исполнителя': ['https://www

In [7]:
df_manual = pd.DataFrame(aggregate_tables)
df_manual

,Ссылка на аккаунт исполнителя,Тип менеджера исполнителя,тип контракта,тг id менеджера исполнителя,Кол-во исполнителей,Дата выхода интеграции,Расходы в Rs,трек ссылка
0,[https://www.instagram.com/zakhmi.writess_?igs...,Блогеры ищеек,Контракт №1,[999999999999],1,[3/25/2024 13:40:08],[7864287],[https://click.tracktik24.com/DKyNxpZ4?landing...
1,NaN,Агентства,Контракт №1,[77777777],[1672],NaN,[1800000],[ссылка нексус 1]
2,NaN,Агентства,Контракт №1,[8888888888],[1089],NaN,[786890],[ссылка рахул 1]
3,[https://www.instagram.com/the_challengers07?i...,Крупные интеграции,Контракт №1,[10000000],1,[22.05.2024],[9878759],[https://click.tracktik24.com/DKyNxpZ4?landing...
4,[https://www.instagram.com/pahadi_vlogger22?ig...,Блогеры ищеек,Контракт №2,[9888888888],1,[3/25/2024 17:25:12],[4287],[https://click.tracktik24.com/DKyNxpZ4?landing...
5,NaN,Агентства,Контракт №2,[77777777],[2567],NaN,[7000000],[ссылка нексус 2]


### Загрузка данные в таблицу Total